# Example: Download ERA-5 wind fields

In [ ]:
import os

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr

import ddeq

# coords of Matimba power plant
sources = ddeq.misc.read_point_sources()
lon_o, lat_o = coords = ddeq.misc.get_source_location(sources.sel(source='Matimba'))
area = [lat_o - 0.251, lon_o - 0.251, lat_o + 0.251, lon_o + 0.251]

## Download different ERA-5 data 
The following function download ERA-5 fields from the reanalysis product either on model or pressure levels, **if** the data not already exists in `ddeq.DATA_PATH` (which should be the case). You can modify the code to download additional data. It is highly recommened to change the data path.

**(Downloading ERA-5 data requires setting up access for the Copernicus Climate Data Store (CDS) Application Program Interface (API). Link: https://cds.climate.copernicus.eu/api-how-to.)**

Download ERA-5 on model levels (normally very slow):

In [ ]:
era5_lvl_filename, era5_sfc_filename = ddeq.era5.download_model_lvl(
    time=pd.Timestamp('2015-04-01 12:00'),
    area=area, prefix='Matimba',
    data_path=ddeq.DATA_PATH
)

Download wind fields on ERA-5 pressure levels

In [ ]:
era5_pressure_filename = ddeq.era5.download_pressure_lvl(
    time=pd.Timestamp('2015-04-01 12:00'),
    area=area, prefix='Matimba',
    data_path=ddeq.DATA_PATH,
    timesteps=1
)

Download PBL height

In [ ]:
era5_pbl_filename = ddeq.era5.download_pbl(
    time=pd.Timestamp('2015-04-01 12:00'),
    area=area, prefix='Matimba',
    data_path=ddeq.DATA_PATH,
    timesteps=1
)

## Vertical averaging
The following function convert 3D model fields to wind speed and direction at source used, for example, by the CSF method.

In [ ]:
# Weighted by GNFR-A emission profiles
gnfra = ddeq.era5.prepare(
    time=pd.Timestamp('2015-04-01 12:00'),
    coords=coords,
    query='gnfra',
    prefix='Matimba',
    data_path=ddeq.DATA_PATH
)

gnfra.speed

In [ ]:
# Mean value in planet boundary layer
pbl = ddeq.era5.prepare(
    time=pd.Timestamp('2015-04-01 12:00'),
    coords=coords,
    query='pbl_mean',
    prefix='Matimba',
    data_path=ddeq.DATA_PATH
)

In [ ]:
# Mean of pressure levels 700-1000 hPa
pres = ddeq.era5.prepare(
    time=pd.Timestamp('2015-04-01 12:00'),
    coords=coords,
    query='pressure_levels',
    prefix='Matimba',
    data_path=ddeq.DATA_PATH
)

## Visualize examples
Plot wind profile for Matimba example.

In [ ]:
era5 = ddeq.era5.compute_height_levels(era5_lvl_filename, era5_sfc_filename)
era5_weighted = ddeq.era5.calculate_gnfra_weighted_winds(era5_lvl_filename, era5_sfc_filename)

In [ ]:
h = era5.h.sel(longitude=lon_o, latitude=lat_o, method='nearest').isel(time=0)
u = era5.u.sel(longitude=lon_o, latitude=lat_o, method='nearest').isel(time=0)
v = era5.v.sel(longitude=lon_o, latitude=lat_o, method='nearest').isel(time=0)
p = era5.p_mid.sel(longitude=lon_o, latitude=lat_o, method='nearest').isel(time=0)

In [ ]:
h = era5.h.mean(['time', 'latitude', 'longitude'])
u = era5.u.mean(['time', 'latitude', 'longitude'])
v = era5.v.mean(['time', 'latitude', 'longitude'])
p = era5.p_mid.mean(['time', 'latitude', 'longitude'])

speed = np.sqrt(u**2 + v**2)
weights = era5_weighted.weights.mean(['time', 'latitude', 'longitude'])

In [ ]:
plt.figure(figsize=(3,6))
plt.plot(speed, h, label='Wind speed')
plt.xlabel('Wind speed [m/s]')
plt.ylabel('Height [m]')

plt.vlines([float(gnfra.speed)], 0, 4500, label='GNFR-A', colors='C1')
plt.vlines([float(pbl.speed)], 0, 4500, label='PBL', colors='C2')
plt.vlines([float(pres.speed)], 0, 4500, label='pressure levels', colors='C3')
plt.vlines(gnfra.isel(time=0).speed.values.flatten(), 0, 4000)

plt.xlim(0,8)
plt.ylim(0,4500)
plt.legend()

plt.twiny()
plt.plot(weights[:], h, 'k--')
plt.xlabel('Weight')